In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\ramak\\OneDrive\\Desktop\\P2\\CatDogClassification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    learning_rate: float
    classes: int
    input_shape: list

In [4]:
from src.CatDogClassification.constants import *
from src.CatDogClassification.utils.common import read_yaml, create_directory

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath = Config_File_Path, param_filepath = Params_File_Path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directory([self.config.main_dir])

    def get_prepared_base_model(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directory([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,
            base_model_path = config.base_model_path,
            updated_base_model_path = config.updated_base_model_path,
            learning_rate = self.params.LEARNING_RATE,
            classes = self.params.CLASSES,
            input_shape = self.params.INPUT_SHAPE
        )

        return prepare_base_model_config


In [6]:
import os
from src.CatDogClassification import logger
import tensorflow as tf
from tensorflow import keras
from keras import layers, models


[2023-12-14 13:18:18,209: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model = models.Sequential()
        self.input_shape = self.config.input_shape


    @staticmethod
    def prepare_base_model(model,input_shape,freeze_all,freeze_till,learning_rate):
        
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape = input_shape))
        model.add(layers.MaxPooling2D((2,2)))

        model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape = input_shape))
        model.add(layers.MaxPooling2D((2,2)))

        model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape = input_shape))
        model.add(layers.MaxPooling2D((2,2)))

        model.add(layers.Flatten())
        model.add(layers.Dense(512, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                           loss = 'binary_crossentropy',
                           metrics = ['accuracy'])

        model.summary()

        return model
    
    def updated_base_model(self):
        self.model = self.prepare_base_model(
            model = self.model,
            input_shape = self.input_shape,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.learning_rate
        )

        self.save_model(path = self.config.updated_base_model_path, model = self.model)

    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)
        logger.info("Saving model")



In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepared_base_model()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.updated_base_model()
except Exception as e:
    raise e

[2023-12-14 13:18:21,359: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\CatDogClassification\config\config.yaml loaded]
[2023-12-14 13:18:21,365: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\CatDogClassification\params.yaml loaded]
[2023-12-14 13:18:21,368: INFO: common: Created directory at: artifacts]
[2023-12-14 13:18:21,372: INFO: common: Created directory at: artifacts/prepare_base_model]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                          

C:\Users\ramak\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2023-12-14 13:18:26,709: INFO: 460451639: Saving model]
